In [3]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
import timeit
from numba import njit

In [2]:
# Create a sample pandas Series
np.random.seed(0)
series = pd.Series(np.random.randn(10000))
window = 50

# Benchmark numpy vs pandas

In [6]:
# Rolling mean
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).mean()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.mean(sliding_window_view(series, window), axis=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.028297 seconds, Std Dev: 0.003984 seconds
Numpy Mean Time : 0.018500 seconds, Std Dev: 0.000188 seconds


In [7]:
# Rolling std
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).std()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.std(sliding_window_view(series, window), axis=1, ddof=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.030795 seconds, Std Dev: 0.004558 seconds
Numpy Mean Time : 0.234323 seconds, Std Dev: 0.024244 seconds


In [8]:
# Rolling median
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    return series.rolling(window=window).median()

# 2. Numpy Implementation
def numpy_rolling(series, window):
    return np.median(sliding_window_view(series, window), axis=1)

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

np.testing.assert_allclose(
    pandas_rolling(series, window).dropna(),
    numpy_rolling(series.values, window)
)

Pandas Mean Time: 0.400690 seconds, Std Dev: 0.010012 seconds
Numpy Mean Time : 0.699383 seconds, Std Dev: 0.010848 seconds


In [9]:
# Rolling multiple stats
# ==============================================================================
# 1. Pandas Implementation
def pandas_rolling(series, window):
    rolling = series.rolling(window=window)
    stats = [rolling.mean(), rolling.std(), rolling.median(), rolling.max(), rolling.min(), rolling.sum()]
    return stats

# 2. Nunmpy Implementation
def numpy_rolling(series, window):

    rolling = sliding_window_view(series, window)
    stats = [
        np.mean(rolling, axis=1),
        np.std(rolling, axis=1, ddof=1),
        np.median(rolling, axis=1),
        np.max(rolling, axis=1),
        np.min(rolling, axis=1),
        np.sum(rolling, axis=1)
    ]
    return stats

repeats = 5
number = 100
pandas_times = timeit.repeat(lambda: pandas_rolling(series, window), number=number, repeat=repeats)
numpy_times = timeit.repeat(lambda: numpy_rolling(series.to_numpy(), window), number=number, repeat=repeats)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")

Pandas Mean Time: 0.526125 seconds, Std Dev: 0.012349 seconds
Numpy Mean Time : 1.083005 seconds, Std Dev: 0.030257 seconds


# Optimization with numba

Durante el predict recursivo, se tiene que aplicar una función repetitiva sobre un array de numpy.

In [18]:
predictions = [np.random.randn(50) for _ in range(1000)]
predictions_pd = [pd.Series(np.random.randn(50)) for _ in range(1000)]

def numpy_version(predictions):
    for last_window in predictions:
        np.median(last_window)

@njit
def numpy_numba_version(predictions):
    for last_window in predictions:
        np.median(last_window)

def pandas_version(predictions):
    for last_window in predictions:
        pd.Series(last_window).median()

repeats = 5
number = 10
numpy_times = timeit.repeat(lambda: numpy_version(predictions), number=number, repeat=repeats)
numba_times = timeit.repeat(lambda: numpy_numba_version(predictions), number=number, repeat=repeats)
pandas_times = timeit.repeat(lambda: pandas_version(predictions_pd), number=number, repeat=repeats)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
numba_mean = np.mean(numba_times)
numba_std = np.std(numba_times)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")
print(f"Numba Mean Time : {numba_mean:.6f} seconds, Std Dev: {numba_std:.6f} seconds")
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")

Numpy Mean Time : 0.154755 seconds, Std Dev: 0.004867 seconds
Numba Mean Time : 0.110940 seconds, Std Dev: 0.072713 seconds
Pandas Mean Time: 0.607512 seconds, Std Dev: 0.004432 seconds


In [22]:
predictions = [np.random.randn(50) for _ in range(1000)]
predictions_pd = [pd.Series(np.random.randn(50)) for _ in range(1000)]

def numpy_version(predictions):
    for last_window in predictions:
        stats = [
            np.mean(last_window),
            np.std(last_window),
            np.median(last_window),
            np.max(last_window),
            np.min(last_window),
            np.sum(last_window)
        ]

@njit
def numpy_numba_version(predictions):
    for last_window in predictions:
        stats = [
            np.mean(last_window),
            np.std(last_window),
            np.median(last_window),
            np.max(last_window),
            np.min(last_window),
            np.sum(last_window)
        ]

def pandas_version(predictions):
    for last_window in predictions:
        lw = pd.Series(last_window)
        stats = [
            lw.mean(),
            lw.std(),
            lw.median(),
            lw.max(),
            lw.min(),
            lw.sum()
        ]

repeats = 5
number = 10
numpy_times = timeit.repeat(lambda: numpy_version(predictions), number=number, repeat=repeats)
numba_times = timeit.repeat(lambda: numpy_numba_version(predictions), number=number, repeat=repeats)
pandas_times = timeit.repeat(lambda: pandas_version(predictions_pd), number=number, repeat=repeats)
numpy_mean = np.mean(numpy_times)
numpy_std = np.std(numpy_times)
numba_mean = np.mean(numba_times)
numba_std = np.std(numba_times)
pandas_mean = np.mean(pandas_times)
pandas_std = np.std(pandas_times)
print(f"Numpy Mean Time : {numpy_mean:.6f} seconds, Std Dev: {numpy_std:.6f} seconds")
print(f"Numba Mean Time : {numba_mean:.6f} seconds, Std Dev: {numba_std:.6f} seconds")
print(f"Pandas Mean Time: {pandas_mean:.6f} seconds, Std Dev: {pandas_std:.6f} seconds")

Numpy Mean Time : 0.464398 seconds, Std Dev: 0.016344 seconds
Numba Mean Time : 0.165382 seconds, Std Dev: 0.188210 seconds
Pandas Mean Time: 1.486151 seconds, Std Dev: 0.015396 seconds


In [55]:
series = pd.Series(np.array([np.nan, 2, np.nan, 4, 5, 6, 7, 8, 9, 10]))
series_2 = pd.Series(np.array([1, 2, 3, 4, 5, np.nan, 7, 8, 9, 10]))

In [56]:
pd.DataFrame({'1':series,
              '2': series_2
}).rolling(window=3, min_periods=3, closed='left').mean()

,1,2
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,2.0
4,NaN,3.0
5,NaN,4.0
6,5.0,NaN
7,6.0,NaN
8,7.0,NaN
9,8.0,8.0


In [ ]:
RollingFeatures(
    stats = str, [str],
    window_sizes = int, [int],
    features_names = None, list,
    min_periods=None=window_size, int, [int],
    closed='left', [str],
    # fill_strategy=ffil, [], #warning, ffill no esta disponible durante predict, usa el argument fill_strategy_predict
    # fill_strategy_predict=None, [],
)

transform_batch
transform



# Skforecast

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [33]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.preprocessing import TimeSeriesDifferentiator

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y as y_categorical
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog as exog_categorical
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation

from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing.tests.fixtures_preprocessing import X

In [30]:
rolling = RollingFeatures(
    stats = ['mean', 'std', 'median', 'max', 'min', 'sum', 'mean'],
    window_sizes = [5, 6, 7, 6, 5, 5, 10],
    # min_periods = [5, 5, 5, 5, 5, 4]
)

In [31]:
print('Stats: ', rolling.stats)
print('Number of stats: ', rolling.n_stats)
print('Window sizes: ', rolling.window_sizes)
print('Max window size: ', rolling.max_window_size)
print('Min periods: ', rolling.min_periods)
print('Features names: ', rolling.features_names)
print('Fill strategy: ', rolling.fill_strategy)

Stats:  ['mean', 'std', 'median', 'max', 'min', 'sum', 'mean']
Number of stats:  7
Window sizes:  [5, 6, 7, 6, 5, 5, 10]
Max window size:  10
Min periods:  [5, 6, 7, 6, 5, 5, 10]
Features names:  ['roll_mean_5', 'roll_std_6', 'roll_median_7', 'roll_max_6', 'roll_min_5', 'roll_sum_5', 'roll_mean_10']
Fill strategy:  None


In [33]:
import pprint
pprint.pp(rolling.unique_rolling_windows)

{'5_5': {'params': {'window': 5, 'min_periods': 5},
         'stats_idx': [0, 4, 5],
         'stats_names': ['roll_mean_5', 'roll_min_5', 'roll_sum_5'],
         'rolling_obj': None},
 '6_6': {'params': {'window': 6, 'min_periods': 6},
         'stats_idx': [1, 3],
         'stats_names': ['roll_std_6', 'roll_max_6'],
         'rolling_obj': None},
 '7_7': {'params': {'window': 7, 'min_periods': 7},
         'stats_idx': [2],
         'stats_names': ['roll_median_7'],
         'rolling_obj': None},
 '10_10': {'params': {'window': 10, 'min_periods': 10},
           'stats_idx': [6],
           'stats_names': ['roll_mean_10'],
           'rolling_obj': None}}


In [39]:
series = pd.Series(np.arange(15), name='series')
# series.iloc[7] = np.nan

df_roll = rolling.transform_batch(series)
df_roll

,roll_mean_5,roll_std_6,roll_median_7,roll_max_6,roll_min_5,roll_sum_5,roll_mean_10
9,7.0,1.870829,6.0,9.0,5.0,35.0,4.5
10,8.0,1.870829,7.0,10.0,6.0,40.0,5.5
11,9.0,1.870829,8.0,11.0,7.0,45.0,6.5
12,10.0,1.870829,9.0,12.0,8.0,50.0,7.5
13,11.0,1.870829,10.0,13.0,9.0,55.0,8.5
14,12.0,1.870829,11.0,14.0,10.0,60.0,9.5


In [43]:
type(rolling.unique_rolling_windows['5_5']['rolling_obj'])

pandas.core.window.rolling.Rolling

In [47]:
df_roll.fillna(df_roll.mean())

,roll_mean_5,roll_std_6,roll_median_7,roll_max_6,roll_min_5,roll_sum_5
0,7.0,1.870829,7.0,9.5,5.0,35.0
1,7.0,1.870829,7.0,9.5,5.0,35.0
2,7.0,1.870829,7.0,9.5,5.0,35.0
3,7.0,1.870829,7.0,9.5,5.0,35.0
4,2.0,1.870829,7.0,9.5,0.0,10.0
5,3.0,1.870829,7.0,5.0,1.0,15.0
6,4.0,1.870829,3.0,6.0,2.0,20.0
7,7.0,1.870829,7.0,9.5,5.0,35.0
8,7.0,1.870829,7.0,9.5,5.0,35.0
9,7.0,1.870829,7.0,9.5,5.0,35.0


In [46]:
df_roll.ffill()

,roll_mean_5,roll_std_6,roll_median_7,roll_max_6,roll_min_5,roll_sum_5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN,0.0,10.0
5,3.0,1.870829,NaN,5.0,1.0,15.0
6,4.0,1.870829,3.0,6.0,2.0,20.0
7,4.0,1.870829,3.0,6.0,2.0,20.0
8,4.0,1.870829,3.0,6.0,2.0,20.0
9,4.0,1.870829,3.0,6.0,2.0,20.0


In [34]:
df_roll['roll_mean_5'].fillna(df_roll['roll_mean_5'].median())

6      4.0
7     10.5
8     10.5
9     10.5
10    10.5
11    10.5
12    10.0
13    11.0
14    12.0
Name: roll_mean_5, dtype: float64

In [35]:
df_roll['roll_mean_5'].bfill()

6      4.0
7     10.0
8     10.0
9     10.0
10    10.0
11    10.0
12    10.0
13    11.0
14    12.0
Name: roll_mean_5, dtype: float64

In [31]:
df_roll['roll_mean_5'].fillna(method='ffill')

C:\Users\jaesc2\AppData\Local\Temp\ipykernel_29004\3132547451.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_roll['roll_mean_5'].fillna(method='ffill')


6      4.0
7      4.0
8      4.0
9      4.0
10     4.0
11     4.0
12    10.0
13    11.0
14    12.0
Name: roll_mean_5, dtype: float64

In [39]:
df_roll['roll_mean_5'].fillna(1.)

6      4.0
7      1.0
8      1.0
9      1.0
10     1.0
11     1.0
12    10.0
13    11.0
14    12.0
Name: roll_mean_5, dtype: float64

In [ ]:
rolling = RollingFeatures(
    stats = ['mean', 'std', 'median', 'max', 'min', 'sum'],
    window_sizes = [5, 6, 7, 6, 5, 5],
    # min_periods = [5, 5, 5, 5, 5, 4]
)

In [58]:
@njit
def _np_min_max_ratio_jit(x):
    return np.min(x) / np.max(x)

In [54]:
@njit
def _np_mean_jit(x):
    return np.mean(x)

In [50]:
@njit
def _np_cv_jit(x):
    a_a, b_b = 0, 0
    for i in x:
        a_a = a_a + i
        b_b = b_b + i * i
    var = b_b / (len(x)) - ((a_a / (len(x))) ** 2)
    var = var * (len(x) / (len(x) - 1))
    std = np.sqrt(var)

    return std / np.mean(x)

In [52]:
arr = np.arange(10000)

In [56]:
%%timeit

for _ in range(1000):
    _np_cv_jit(arr)

13 ms ± 40.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%%timeit

for _ in range(1000):
    _np_mean_jit(arr)

9.54 ms ± 7.74 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
%%timeit

for _ in range(1000):
    _np_min_max_ratio_jit(arr)

3.92 ms ± 137 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
arr = np.arange(10000)

In [5]:
rolling = RollingFeatures(
    stats = ['mean', 'std', 'median', 'max', 'min', 'sum'],
    window_sizes = [10, 10, 10, 10, 10, 10],
)

In [6]:
%%timeit

for _ in range(1000):
    rolling.transform(arr)

14.9 ms ± 614 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
rolling = RollingFeatures(
    stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 'ratio_min_max', 'coef_variation'],
    window_sizes = [10, 10, 10, 10, 10, 10, 10, 10],
)

In [8]:
%%timeit

for _ in range(1000):
    rolling.transform(arr)

16.8 ms ± 902 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
series = pd.Series(np.arange(10000), name='y')

In [17]:
df_1 = rolling.transform_batch(series)
df_2 = rolling.transform_batch_2(series)

pd.testing.assert_frame_equal(df_1, df_2)

In [18]:
df_2.head()

,roll_mean_10,roll_std_10,roll_min_10,roll_max_10,roll_sum_10,roll_median_10,roll_ratio_min_max_10,roll_coef_variation_10
9,4.5,3.02765,0.0,9.0,45.0,4.5,0.000000,0.672811
10,5.5,3.02765,1.0,10.0,55.0,5.5,0.100000,0.550482
11,6.5,3.02765,2.0,11.0,65.0,6.5,0.181818,0.465792
12,7.5,3.02765,3.0,12.0,75.0,7.5,0.250000,0.403687
13,8.5,3.02765,4.0,13.0,85.0,8.5,0.307692,0.356194


In [16]:
%%timeit

rolling.transform_batch_2(series).to_numpy()

5.5 ms ± 129 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 
             'ratio_min_max', 'coef_variation']
    
rolling = RollingFeatures(stats=stats, window_sizes=10)
X_window = X.iloc[-10:]

In [9]:
rolling_obj = X_window.rolling(**rolling.unique_rolling_windows['10_10']['params'])
rolling_obj.mean().iat[-1]

np.float64(0.5189421270000001)

In [13]:
stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 
             'ratio_min_max', 'coef_variation']
    
rolling = RollingFeatures(stats=stats, window_sizes=4)

rolling.transform_batch(X).head(10)

,roll_mean_4,roll_std_4,roll_min_4,roll_max_4,roll_sum_4,roll_median_4,roll_ratio_min_max_4,roll_coef_variation_4
3,0.440194,0.221565,0.226851,0.696469,1.760775,0.418727,0.325716,0.503334
4,0.445944,0.230549,0.226851,0.719469,1.783775,0.418727,0.315304,0.516990
5,0.480185,0.207968,0.226851,0.719469,1.920742,0.487211,0.315304,0.433099
6,0.668664,0.240871,0.423106,0.980764,2.674654,0.635392,0.431405,0.360228
7,0.702042,0.228102,0.423106,0.980764,2.808169,0.702149,0.431405,0.324911
8,0.642408,0.251960,0.423106,0.980764,2.569632,0.582881,0.431405,0.392212
9,0.634661,0.261256,0.392118,0.980764,2.538643,0.582881,0.399808,0.411647
10,0.475264,0.150897,0.343178,0.684830,1.901057,0.436525,0.501114,0.317502
11,0.486319,0.171572,0.343178,0.729050,1.945277,0.436525,0.470720,0.352796
12,0.475729,0.173313,0.343178,0.729050,1.902917,0.415345,0.470720,0.364311


In [14]:
rolling.transform_batch(X).head(10).to_numpy()

array([[0.44019369, 0.22156465, 0.22685145, 0.69646919, 1.76077474,
        0.41872705, 0.32571642, 0.50333446],
       [0.44594363, 0.23054861, 0.22685145, 0.71946897, 1.78377452,
        0.41872705, 0.31530401, 0.51699048],
       [0.48018541, 0.20796803, 0.22685145, 0.71946897, 1.92074165,
        0.48721061, 0.31530401, 0.43309944],
       [0.6686636 , 0.24087135, 0.42310646, 0.9807642 , 2.6746544 ,
        0.63539187, 0.43140488, 0.360228  ],
       [0.70204234, 0.22810163, 0.42310646, 0.9807642 , 2.80816937,
        0.70214935, 0.43140488, 0.3249115 ],
       [0.64240807, 0.25196046, 0.42310646, 0.9807642 , 2.5696323 ,
        0.58288082, 0.43140488, 0.39221247],
       [0.63466084, 0.26125613, 0.39211752, 0.9807642 , 2.53864336,
        0.58288082, 0.39980815, 0.41164685],
       [0.4752643 , 0.15089728, 0.34317802, 0.68482974, 1.90105718,
        0.43652471, 0.50111436, 0.31750182],
       [0.48631929, 0.17157163, 0.34317802, 0.72904971, 1.94527715,
        0.43652471, 0.470719

In [24]:
X_datetime = X.copy()
X_datetime.index = pd.date_range(start='1990-01-01', periods=len(X), freq='D')
X_datetime.iloc[9] = np.nan

stats = ['mean', 'std', 'mean', 'max']
window_sizes = [4, 5, 6, 4]
min_periods = [3, 5, 6, 3]
features_names = ['my_mean', 'my_std', 'my_mean_2', 'my_max']

rolling = RollingFeatures(
    stats=stats, window_sizes=window_sizes, min_periods=min_periods,
    features_names=features_names, fillna=5.
)
rolling_features = rolling.transform_batch(X_datetime).head(15)
rolling_features

,my_mean,my_std,my_mean_2,my_max
1990-01-06,0.480185,0.199922,0.483892,0.719469
1990-01-07,0.668664,0.287322,0.531274,0.980764
1990-01-08,0.702042,0.208726,0.597723,0.980764
1990-01-09,0.642408,0.220909,0.640069,0.980764
1990-01-10,0.715509,5.000000,5.000000,0.980764
1990-01-11,0.502980,5.000000,5.000000,0.684830
1990-01-12,0.517720,5.000000,5.000000,0.729050
1990-01-13,0.503600,5.000000,5.000000,0.729050
1990-01-14,0.392619,5.000000,5.000000,0.729050
1990-01-15,0.406336,0.239069,5.000000,0.729050


In [60]:
stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 
             'ratio_min_max', 'coef_variation']
    
rolling = RollingFeatures(stats=stats, window_sizes=4)
rolling_features = rolling.transform(X.to_numpy())
rolling_features

0 mean
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343        nan        nan        nan        nan        nan
        nan        nan]
1 std
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343 0.23013666        nan        nan        nan        nan
        nan        nan]
2 min
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343 0.23013666 0.48303426        nan        nan        nan
        nan        nan]
3 max
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343 0.23013666 0.48303426 0.98555979        nan        nan
        nan        nan]
4 sum
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343 0.23013666 0.48303426 0.98555979 2.6009737         nan
        nan        nan]
5 median
X_window [0.48303426 0.98555979 0.51948512 0.61289453]
RF: [0.65024343 0.23013666 0.48303426 0.98555979 2.6009737  0.56618983
        nan        nan]
6 ratio_min_max
X_window [0.48303426 0.98555979 0.51948512 0

array([0.65024343, 0.23013666, 0.48303426, 0.98555979, 2.6009737 ,
       0.56618983, 0.49011157, 0.35392385])

In [27]:
X.to_numpy()[-4:].mean()

np.float64(0.6502434250000001)

In [67]:
X = pd.Series(
    data = np.array(
                [0.69646919, 0.28613933, 0.22685145, 0.55131477, 0.71946897,
                 0.42310646, 0.9807642 , 0.68482974, 0.4809319 , 0.39211752,
                 0.34317802, 0.72904971, 0.43857224, 0.0596779 , 0.39804426,
                 0.73799541, 0.18249173, 0.17545176, 0.53155137, 0.53182759,
                 0.63440096, 0.84943179, 0.72445532, 0.61102351, 0.72244338,
                 0.32295891, 0.36178866, 0.22826323, 0.29371405, 0.63097612,
                 0.09210494, 0.43370117, 0.43086276, 0.4936851 , 0.42583029,
                 0.31226122, 0.42635131, 0.89338916, 0.94416002, 0.50183668,
                 0.62395295, 0.1156184 , 0.31728548, 0.41482621, 0.86630916,
                 0.25045537, 0.48303426, 0.98555979, 0.51948512, 0.61289453
           ]),
    name = 'X'
)

X_nans = X.tail(15).to_numpy()
X_nans[-7] = np.nan

stats = ['mean', 'std', 'min', 'max', 'sum', 'median', 
            'ratio_min_max', 'coef_variation']
window_sizes = [10, 10, 15, 4, 15, 4, 4, 4]
min_periods = [1, 1, 1, 1, 1, 1, 1, 1]
rolling = RollingFeatures(stats=stats, window_sizes=window_sizes, min_periods=min_periods)
rolling_features = rolling.transform(X_nans)
rolling_features

array([0.53051056, 0.28145959, 0.1156184 , 0.98555979, 7.85259345,
       0.56618983, 0.49011157, 0.35392385])

In [44]:
X_nans_pandas = X.copy()
X_nans_pandas.iloc[-7] = np.nan

rolling_features = rolling.transform_batch(X_nans_pandas)
rolling_features.tail()

,roll_mean_10,roll_std_10,roll_min_15,roll_max_4,roll_sum_4,roll_median_4,roll_ratio_min_max_4,roll_coef_variation_4
45,0.548818,0.301862,0.115618,0.866309,1.434050,0.317285,0.289106,0.706936
46,0.555116,0.299570,0.115618,0.866309,1.599799,0.483034,0.289106,0.583169
47,0.565357,0.313817,0.115618,0.985560,2.585359,0.674672,0.254125,0.526077
48,0.518171,0.279826,0.115618,0.985560,2.238535,0.501260,0.254125,0.550255
49,0.530511,0.281460,0.115618,0.985560,2.600974,0.566190,0.490112,0.353924


In [46]:
X_nans_pandas.tail(10)

40    0.623953
41    0.115618
42    0.317285
43         NaN
44    0.866309
45    0.250455
46    0.483034
47    0.985560
48    0.519485
49    0.612895
Name: X, dtype: float64

In [47]:
X_nans_pandas.tail(10).mean()

np.float64(0.5305105622222223)

In [48]:
X_nans_pandas.tail(10).to_numpy().mean()

np.float64(nan)

In [49]:
np.nanmean(X_nans_pandas.tail(10).to_numpy())

np.float64(0.5305105622222223)

In [66]:
np.nansum(X_nans_pandas.tail(15).to_numpy())

np.float64(7.8525934500000005)

In [51]:
X_nans_pandas.tail(10).to_numpy()[~np.isnan(X_nans_pandas.tail(10).to_numpy())]

array([0.62395295, 0.1156184 , 0.31728548, 0.86630916, 0.25045537,
       0.48303426, 0.98555979, 0.51948512, 0.61289453])

In [52]:
X_nans_pandas.tail(10).to_numpy()[~np.isnan(X_nans_pandas.tail(10).to_numpy())].mean()

np.float64(0.5305105622222221)

In [53]:
np.mean(X_nans_pandas.tail(10).to_numpy()[~np.isnan(X_nans_pandas.tail(10).to_numpy())])

np.float64(0.5305105622222221)

In [96]:
X_datetime = X.head(10).copy()
X_datetime.index = pd.date_range(start='1990-01-01', periods=len(X_datetime), freq='D')
X_datetime.iloc[6] = np.nan

rolling = RollingFeatures(
    stats=['mean'], window_sizes=3, fillna='median'
)
rolling_features = rolling.transform_batch(X_datetime)
rolling_features

,roll_mean_3
1990-01-03,0.403153
1990-01-04,0.354769
1990-01-05,0.499212
1990-01-06,0.564630
1990-01-07,0.499212
1990-01-08,0.499212
1990-01-09,0.499212
1990-01-10,0.519293


In [97]:
rolling_features.to_numpy().flatten()

array([0.40315332, 0.35476852, 0.49921173, 0.56463007, 0.49921173,
       0.49921173, 0.49921173, 0.51929305])

In [80]:
d = {'1': 1, 0: 1}
d

{'1': 1, 0: 1}

In [87]:
base_array = np.array([0.40315332, 0.35476852, 0.49921173, 0.56463007, 
                           0.70777988, np.nan, np.nan, np.nan])
mean_ = np.array([0.5059087, 0.5059087, 0.5059087])

In [88]:
base_array[-3:] = mean_
base_array

array([0.40315332, 0.35476852, 0.49921173, 0.56463007, 0.70777988,
       0.5059087 , 0.5059087 , 0.5059087 ])

In [98]:
base_array = np.array([0.40315332, 0.35476852, 0.49921173, 0.56463007, 
                        np.nan, np.nan, np.nan, 0.519293])
aa = np.array([5., 5., 5.])

In [99]:
expected_array = base_array
expected_array[-4:-1] = aa
expected_array

array([0.40315332, 0.35476852, 0.49921173, 0.56463007, 5.        ,
       5.        , 5.        , 0.519293  ])